In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed


def run_thread_pool_sub(target, args, max_work_count=3):
    with ThreadPoolExecutor(max_workers=max_work_count) as t:
        res = [t.submit(target, i) for i in args]
        return res


In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi

# 设置参数
S0 = 1      # 初始资产价格
T = 35.0    # 总时长（例如1年）
r = 0.06    # 风险无关利率
l = 0.0125  # 风险费率
sigma = 0.2 # 波动率
dt = 0.01   # 时间步长
np.random.seed(7)  # 设置固定的随机数种子

M = 200
g = 0
lamda = 1/35

def stock_SDE(S0, r, sigma, dt, t, l):
    N = int(t / dt)
    if N == 0:
        return S0
    
    S = np.zeros(N)
    S[0] = S0
    for i in range(1, N):
        Z = np.random.normal(0, 1)
        S[i] = S[i-1] * np.exp((r - l - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return S[-1]

def average_payoff(g, t, S0, r, sigma, dt, l):
    S_t = stock_SDE(S0, r, sigma, dt, t, l)  # 使用时间t调用stock_SDE
    return max(math.exp(g * t) - S_t, 0)

def f(t):
    single_V = lamda * math.exp(-lamda*t) * math.exp(-r*t) * average_payoff(g, t, S0, r, sigma, dt, l)
    return single_V

def int_death_payoff(f, a, b, limit=100):
    # 计算定积分，从a到b，增加子区间限制
    result, error = spi.quad(f, a, b, limit=limit)
    print(result)   
    return result

# 分割积分区间
interval_divisions = 5  # 定义区间划分的数量
intervals = np.linspace(0, T, interval_divisions+1)  # 划分区间

def thread_function(i):
    return sum(int_death_payoff(f, intervals[j], intervals[j+1]) for j in range(interval_divisions))


args = list(range(M))
res = run_thread_pool_sub(thread_function, args, max_work_count=10)
print("res:", res)

final_result = []
for future in as_completed(res):
    data = future.result()
    final_result.append(data)

G = sum(final_result)
death_payoff = G / M

print("最终结果：", death_payoff)


/tmp/ipykernel_760/928549457.py:41: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, error = spi.quad(f, a, b, limit=limit)


0.010954805138953182
0.011653390653591568
0.010841337290886219
0.010724234943858057
0.011501797123818682
0.011625861707852415
0.01232612882150067
0.012162369841876299
0.012220671294352992
0.011776745011243315
0.008100213916227084
0.008381111194155284
0.008877480151402455
0.00818361158325861
0.009116799818701227
0.008940719994929762
0.0074654092693395195
0.007885731910365526
0.00832187289719313
0.008046437885734118
0.004363214943665841
0.004277151704355528
0.0038827801149316503
0.004447319960390327
0.004476629820334296
0.004651911027997421
0.004089618997501101
0.004440702005766597
0.004414354046719906
0.004529898824160405
0.002137464802029618
0.002027710741375472
0.0021912654253766263
0.0020857352515258987
0.002315890153685436
0.0027050702241679142
0.002344043731425439
0.0022742293274869707
0.0020864597471552584
0.002621293854076756
0.0009356749164661138
0.0008214733956213516
0.001179678471686543
0.000945369105392224
0.001105063607325905
0.0011354896115579618
0.0010060039156061138
0.001

/tmp/ipykernel_760/928549457.py:41: IntegrationWarning: The maximum number of subdivisions (100) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result, error = spi.quad(f, a, b, limit=limit)


0.0006626085461871426
0.0010240348073422278
0.010512064249860193
0.011214491681977933
0.011246394999709468
0.011715650234385137
0.011587313488757963
0.011548808063328222
0.010661378851942098
0.011125975754436803
0.011547543420151862
0.010414977406252078
0.007850171865457483
0.007558738626962207
0.007519908936960744
0.008830871091526285
0.008665726329680705
0.008865707137975128
0.009942437816627238
0.006543790042868834
0.007417366042414394
0.007532245636031038
0.004150850628386577
0.004492581917433273
0.004484202342699575
0.004408977063999952
0.004046836030081932
0.0034625596561947366
0.0050510713419387645
0.004578355634294335
0.004764491519733034
0.004626785702742565
0.0023910859521803387
0.0020378908943234028
0.00198303369653908
0.0025263330924146874
0.0023279553453500962
0.002261929057343095
0.0015693797057116027
0.002074712829036008
0.002185582705241665
0.002384185778497312
0.001075553058074508
0.0009868894283148674
0.000931642399860934
0.0010523126126006676
0.0012937239487031671
0.